# (1) Read Data

This is copied from [USPPPM] Simple RoBERTa starter (Eng/日本語) with some explanations, explorations,etc. being deleted!
https://www.kaggle.com/code/junjitakeshima/uspppm-simple-roberta-starter-eng go upvote the original! Just easier for me to fork it and edit so I can see what's going on better!

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import transformers
from transformers import RobertaTokenizer, TFRobertaModel
transformers.logging.set_verbosity_error()
import re
pd.set_option("display.max_columns", None)

In [ ]:
train_df = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
test_df  = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
train_df

In [ ]:
train_df["context"].unique()

As xhlulu consolidated hundreds of USTPO categories into a single csv as a public dataset, I'm happy to use it.

xhluluさんが以下のリンク先で、何百ものUSTPOの分野カテゴリーを一つのcsvに纏めてくれたものを公開しているので、有り難く使わせていただく。

https://www.kaggle.com/datasets/xhlulu/cpc-codes

In [ ]:
cpc_codes = pd.read_csv("/kaggle/input/cpc-codes/titles.csv")
cpc_codes

In [ ]:
cpc_codes = cpc_codes.rename(columns = {"code" : "context"})
train_df = pd.merge(train_df, cpc_codes[["context","title"]], on ="context", how = "left")
test_df = pd.merge(test_df, cpc_codes[["context","title"]], on ="context", how = "left")
train_df

In [ ]:
max_len = 128
train_df["title"] = train_df["title"].apply(lambda x : re.sub('[;,]', '', x))
print(train_df.loc[:500, "title"].unique())

In [ ]:
train_df["anchor_title"] = train_df["anchor"].astype(str) + " " + train_df["title"].astype(str)
test_df["anchor_title"] = test_df["anchor"].astype(str) + " " + test_df["title"].astype(str)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("../input/roberta-base/")

In [ ]:
def create_data(id_, anchor_title, target, score, train=True) :
    input_ids = []
    attention_mask = []
    labels = []
    ids = []
    tok_txt = tokenizer.batch_encode_plus(
                           [(word[0], word[1]) for word in zip(anchor_title, target)],                           
                           max_length = max_len, 
                           padding='max_length',
                           truncation=True)    
    for i in range(len(anchor_title)): 
        ids.append(id_[i])
        input_ids.append(tok_txt['input_ids'][i])
        attention_mask.append(tok_txt['attention_mask'][i])
        if train:
            labels.append(score[i])
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "ids":ids, 
            }, labels

In [ ]:
train_data, train_labels = create_data(train_df['id'], train_df['anchor_title'], 
                                       train_df['target'], train_df['score'], train=True)

In [ ]:
test_data, test_labels = create_data(test_df['id'], test_df['anchor_title'], 
                                     test_df['target'], None, train=False)

# (3) Build Model

Thanks to the below-link for pretrained RoBERTa model.  

RoBERTaモデルとしては以下リンク先で公開されている学習済みモデルを使わせていただく。

https://www.kaggle.com/datasets/abhishek/roberta-base

In [ ]:
def build_model():
    
    model_ids = Input(shape=(max_len, ), dtype = tf.int32)
    model_mask = Input(shape=(max_len, ), dtype = tf.int32)
    
    roberta_model = TFRobertaModel.from_pretrained("../input/roberta-base/")
    
    x = roberta_model(input_ids = model_ids, 
                      attention_mask = model_mask)       
    x = tf.keras.layers.GlobalAveragePooling1D()(x.last_hidden_state)    
    outputs = Dense(1)(x)
    
    model = tf.keras.Model(inputs = [model_ids, model_mask], outputs = outputs)
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(), #no idea!
        loss = "mse",
        metrics=["mse"])
    return model

In [ ]:
def scheduler(epoch):
    learning_rate = 2e-5
    if epoch == 0:
        return learning_rate * 0.05
    else:
        return learning_rate * (0.9**epoch)
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model = build_model() #history todo
model.fit((np.array(train_data['input_ids']),
           np.array(train_data['attention_mask'])),
        np.array(train_labels).ravel(), 
        epochs = 10,
        shuffle=True,
        callbacks = [EarlyStopping(monitor='val_mse', patience=3, restore_best_weights=True), 
                     ModelCheckpoint('roberta_uspppm.h5', monitor='val_mse', 
                                     save_best_only=True, save_weights_only=True), 
                     callback_lr],                     
        batch_size = 16,
        validation_split=0.2 )

# (4) Prediction

In [ ]:
test_preds = model.predict((np.array(test_data['input_ids']),
                            np.array(test_data['attention_mask'])))

In [ ]:
submission = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv")
submission['score'] = test_preds
submission['score'] = submission.score.apply(lambda x: 0 if x < 0 else x)
submission['score'] = submission.score.apply(lambda x: 1 if x > 1 else x)
submission.to_csv('submission.csv',index=False)
submission

# (5) Credits

Thanks to below-links;

for RoBERTa model  
https://www.kaggle.com/datasets/abhishek/roberta-base  
for CPC codes datasets  
https://www.kaggle.com/datasets/xhlulu/cpc-codes  
for model configuration  
https://www.kaggle.com/code/samarthagarwal23/0-8-tf-keras-deberta-baseline-single-model?scriptVersionId=91187254